# Setup

In [1]:
############################################################################################################################ 
# Get the latest CREST files for each ID within the target folder (dirname)

from pathlib import Path
import json
from scipy.optimize import curve_fit
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from random import choice as random_choice
from itertools import combinations
from numpy import array, unravel_index, argmin, mean
from sklearn.metrics import pairwise_distances
import random
import numpy as np
from copy import deepcopy
import itertools
from time import time
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from neuprint import skeleton as npskel
from cloudvolume import Skeleton
from io import StringIO
import networkx as nx
import re
from tqdm import tqdm

import sys
sys.path.append('/Users/kperks/Documents/ell-connectome/efish_em/efish_em')

# from eCREST_cli_beta import ecrest, import_settings
from eCREST_cli import ecrest
import AnalysisCode as efish 

The 'ecrest' class has been imported from eCREST_cli.py

An instance of this object will be able to:
- open an neuroglancer viewer for proofrieading (see "Proofread using CREST")
    - add-remove segments (using graph feature for efficiency)
    - format itself and save itself as a CREST-style .json
- convert from neuroglancer json (see "Convert From Neuroglancer to eCREST")
    - format itself and save itself as a CREST-style .json
    


## Import settings

If you save a copy of settings_dict.json (found in the "under construction" directory of eCREST repo) locally somewhere outside the repo (like in your save_dir), then you can use the following code cell to import. This avoids needing to re-type the save_dir and db_path each time you "git pull" updates from the repo to this notebook.

In [2]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = efish.import_settings(path_to_settings_json)

vx_sizes = [16,16,30]

## Load files

In [3]:
dirpath = Path(settings_dict['save_dir'])
# dirpath = "/Users/kperks/Documents/gdrive/.shortcut-targets-by-id/16q1BuOMfD2ta0Cwq8CjMlRe4rDvbuWC5/ELL_connectome/CREST_reconstructions/mg-network"

In [4]:
nodefiles = efish.get_cell_filepaths(dirpath)

# TODO reconstruction files from synapses

In [5]:
syn_to_find = set()
        


syn_type = 'post-synaptic'#'spine_inputs' #

vx_sizes = [16, 16, 30]


In [6]:
# cell_type = {}
# not_typed = []
# for x,f in nodefiles.items():
#     cell = ecrest(settings_dict,filepath = f,launch_viewer=False)
#     cell_type[x] = cell.get_ctype('manual') 
#     if (cell.get_ctype('manual') == []) | (cell.get_ctype('manual') == ''):
#         cell_type[x]=''
#         not_typed.append(x)# print(f'cell {x} is not cell-typed in json')
        
# print('the following cells are not typed in the main network')
# print(not_typed)        
        

pfs reconstructed from each type
299496636 mg1 5/22, 
214581797 mg2 1/22, 
301787806 lg, 11/11
393325331 lf, 8/16

In [6]:
cells_todo = ['36145073'] #[str(c) for c in ['472361842', '387382792', '387197529', '472051969', '386690280']] #[k for k,v in cell_type.items() if v=='pf'] #['393325331']# 

In [7]:
syn_to_find = set()

for c_id in cells_todo:
    cell_data = efish.load_ecrest_celldata(filepath= nodefiles[c_id]) #= ecrest(settings_dict,filepath= nodefiles[c_id], launch_viewer=False)
    for syn_ in cell_data['end_points'][syn_type]:# crest.cell_data['end_points'][syn_type]:
        try:
            syn_to_find.add(syn_[0][3])

        except IndexError as msg:
            cellid = cell_data['metadata']['main_seg']['base']
            print(msg, f'for cell {cellid} synapse at {array([int(syn_[0][i]/vx_sizes[i]) for i in range(3)])} has no segment id')

In [8]:
len(cell_data['end_points'][syn_type])

66

In [9]:
len(syn_to_find)

52

First, find if any of these post-synaptic segments are already part of reconstructions completed

In [10]:
# crest = ecrest(settings_dict,filepath = nodefiles[cells_todo[0]])

base_segments = efish.get_base_segments_dict(Path(settings_dict['save_dir']))#/'todo/sgx2_394470350_pre')#/ 'todo_presynaptic')# / 'todo_afferent')#'todo_postsynaptic_grc') 

In [11]:
topop=set()
for k,v in base_segments.items():
    if syn_to_find & v != set():
        # print(f'use reconstruction {k}')
        topop = topop.union(syn_to_find & v)
    

len(topop)

43

Adjust "syn_to_find" to eliminate these base segments from the todo list

In [12]:
syn_to_find = syn_to_find.difference(topop)

len(syn_to_find)

9

In [13]:
syn_to_find

{'215898220',
 '385341269',
 '385341412',
 '385341878',
 '470072631',
 '471264318',
 '552479953',
 '552512537',
 '555948799'}

create crest files for each of the unidentified post-synaptic partners

Save reconstructed_segs as a json to go through manually

In [17]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = efish.import_settings(path_to_settings_json)


In [18]:
todo_folder_path = Path(settings_dict['save_dir']) / 'todo/pe_post'

for segment_id in sorted(list(syn_to_find)):

    cell = ecrest(settings_dict,segment_id = segment_id, launch_viewer=False)
    cell.save_cell_graph(directory_path = todo_folder_path)#'todo_presynaptic/Krista/sgx_394470350')#/Krista/mg_214581797')

Creating base segment graph for cell 370427755 Cell Reconstruction
all base locations for 1 obtained from SQL database
graph created among all_base_segs
1 clusters of connected components. Connecting these clusters with nearest base segments.
weak clusters connected
segments without a location connected
1 clusters in graph (note should/would be only 1 if loaded base ID from agglomo fresh)
Created a CREST instance for NEW Reconstruction of 370427755. No file saved yet -- save manually.
Saved cell 370427755 reconstruction locally at 2025-10-21 12.48.37
Creating base segment graph for cell 370427758 Cell Reconstruction
all base locations for 1 obtained from SQL database
graph created among all_base_segs
1 clusters of connected components. Connecting these clusters with nearest base segments.
weak clusters connected
segments without a location connected
1 clusters in graph (note should/would be only 1 if loaded base ID from agglomo fresh)
Created a CREST instance for NEW Reconstruction of 